In [2]:
#
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
data=pd.read_parquet('hotel_bookings.parquet')
data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [281]:
data['is_canceled'].unique()

array([0, 1], dtype=int64)

In [274]:
columns=list(data.columns)

for i in range(len(columns)):

    if i>=19:

        print(columns[i],data[columns[i]].dtypes)

reserved_room_type object
assigned_room_type object
booking_changes int64
deposit_type object
agent float64
company float64
days_in_waiting_list int64
customer_type object
adr float64
required_car_parking_spaces int64
total_of_special_requests int64
reservation_status object
reservation_status_date object


In [209]:
# create new features

def reservation_season(date:str):

    season={'winter': ['December', 'January', 'February'],
            'Spring': ['March','April','May'],
            'Summer':['June','July','August'],
            'Autumn':['September','October','November']
    }

    for key, val in season.items():

        if date in val:

            return key
        
def binning_arrival_date_month(date):

    if int(date)<=10:

        return 'begin_month'
    
    elif 10<int(date)<=20:

        return 'Middle_month'
    
    else:

        return 'End_month'
        




In [210]:
data["Arrival_season"]=data['arrival_date_month'].apply(reservation_season)
data['room_has_changed']=np.where(data['assigned_room_type']!=data['reserved_room_type'],1,0)

data['arrival_day_month']=data['arrival_date_day_of_month'].apply(binning_arrival_date_month)




In [4]:
#columns to drop

columns_to_drop=['hotel', 'reservation_status','reservation_status_date','arrival_date_year','agent','company','arrival_date_day_of_month','arrival_date_month']

data.drop(columns=columns_to_drop,axis=1,inplace=True)

In [212]:
data['arrival_date_month'].unique()

KeyError: 'arrival_date_month'

In [213]:
data

,is_canceled,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,...,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,Arrival_season,room_has_changed,arrival_day_month
0,0,342,27,0,0,2,0.0,0,BB,PRT,...,3,No Deposit,0,Transient,0.00,0,0,Summer,0,begin_month
1,0,737,27,0,0,2,0.0,0,BB,PRT,...,4,No Deposit,0,Transient,0.00,0,0,Summer,0,begin_month
2,0,7,27,0,1,1,0.0,0,BB,GBR,...,0,No Deposit,0,Transient,75.00,0,0,Summer,1,begin_month
3,0,13,27,0,1,1,0.0,0,BB,GBR,...,0,No Deposit,0,Transient,75.00,0,0,Summer,0,begin_month
4,0,14,27,0,2,2,0.0,0,BB,GBR,...,0,No Deposit,0,Transient,98.00,0,1,Summer,0,begin_month
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0,23,35,2,5,2,0.0,0,BB,BEL,...,0,No Deposit,0,Transient,96.14,0,0,Summer,0,End_month
119386,0,102,35,2,5,3,0.0,0,BB,FRA,...,0,No Deposit,0,Transient,225.43,0,2,Summer,0,End_month
119387,0,34,35,2,5,2,0.0,0,BB,DEU,...,0,No Deposit,0,Transient,157.71,0,4,Summer,0,End_month
119388,0,109,35,2,5,2,0.0,0,BB,GBR,...,0,No Deposit,0,Transient,104.40,0,0,Summer,0,End_month


In [5]:
numerical_features=[col for col in data.columns if data[col].dtypes!='O']
categorical_features=[col for col in data.columns if data[col].dtypes=='O']

In [6]:
numerical_features

['is_canceled',
 'lead_time',
 'arrival_date_week_number',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'is_repeated_guest',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'booking_changes',
 'days_in_waiting_list',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests']

In [7]:
categorical_features

['meal',
 'country',
 'market_segment',
 'distribution_channel',
 'reserved_room_type',
 'assigned_room_type',
 'deposit_type',
 'customer_type']

In [8]:
data['is_canceled'].unique()

array([0, 1], dtype=int64)

In [9]:
# transformation

ordinal_columns=['customer_type','market_segment','reserved_room_type','distribution_channel','reserved_room_type','assigned_room_type']

oh_columns=[col for col in categorical_features if col not in ordinal_columns]



In [277]:
oh_columns

['meal', 'country', 'deposit_type', 'Arrival_season', 'arrival_day_month']

In [42]:
for col in oh_columns:

    print(col, len(data[col].unique()))

meal 5
country 178
deposit_type 3
Arrival_season 4
arrival_day_month 3


In [43]:
for col in ordinal_columns:

    print(col, len(data[col].unique()))

customer_type 4
market_segment 8
reserved_room_type 10
distribution_channel 5
reserved_room_type 10
assigned_room_type 12


In [10]:
from sklearn.model_selection import train_test_split

df_train,df_test=train_test_split(data,test_size=0.2,random_state=42,stratify=data['is_canceled'],shuffle=True)

In [11]:
X=df_train.drop('is_canceled',axis=1)
y=df_train['is_canceled']

X_test=df_test.drop('is_canceled',axis=1)
y_test=df_test['is_canceled']

In [12]:
numerical_features=[col for col in X.columns if col in numerical_features]

In [109]:

X[numerical_features].dtypes

lead_time                           int64
arrival_date_week_number            int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
booking_changes                     int64
days_in_waiting_list                int64
adr                               float64
required_car_parking_spaces         int64
total_of_special_requests           int64
room_has_changed                    int32
dtype: object

In [13]:
X[oh_columns].dtypes

meal            object
country         object
deposit_type    object
dtype: object

In [14]:
for col in X[ordinal_columns].columns:

    print(col, X[col].unique())

customer_type ['Transient' 'Transient-Party' 'Contract' 'Group']
market_segment ['Online TA' 'Groups' 'Offline TA/TO' 'Corporate' 'Direct' 'Complementary'
 'Aviation' 'Undefined']
reserved_room_type ['A' 'D' 'F' 'G' 'C' 'E' 'B' 'H' 'P' 'L']
distribution_channel ['GDS' 'TA/TO' 'Corporate' 'Direct' 'Undefined']
reserved_room_type ['A' 'D' 'F' 'G' 'C' 'E' 'B' 'H' 'P' 'L']
assigned_room_type ['A' 'D' 'F' 'G' 'C' 'E' 'B' 'H' 'I' 'K' 'P' 'L']


In [129]:
X

,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,...,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,Arrival_season,room_has_changed,arrival_day_month
105447,20,6,0,2,1,0.0,0,BB,IRL,Online TA,...,0,No Deposit,0,Transient,98.00,0,0,winter,0,begin_month
85242,8,11,0,3,2,0.0,0,BB,FRA,Groups,...,0,No Deposit,0,Transient-Party,100.00,0,0,Spring,0,begin_month
65604,93,14,0,3,2,0.0,0,BB,PRT,Offline TA/TO,...,0,Non Refund,0,Transient,95.00,0,0,Spring,0,begin_month
17345,90,39,4,10,2,0.0,0,BB,GBR,Offline TA/TO,...,0,No Deposit,0,Contract,54.00,0,0,Autumn,0,End_month
117786,8,32,0,1,1,0.0,0,BB,ESP,Corporate,...,0,No Deposit,0,Transient,80.00,0,0,Summer,0,begin_month
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104906,22,3,2,4,1,0.0,0,BB,EST,Online TA,...,0,No Deposit,0,Transient,58.24,0,2,winter,0,Middle_month
50818,18,20,1,2,1,0.0,0,BB,RUS,Online TA,...,0,No Deposit,0,Transient,136.00,0,0,Spring,0,begin_month
36939,135,22,0,3,2,0.0,0,BB,CN,Online TA,...,0,No Deposit,0,Transient,94.97,0,2,Spring,0,End_month
14587,172,24,2,5,2,0.0,0,BB,GBR,Direct,...,0,No Deposit,0,Transient,113.40,0,1,Summer,0,Middle_month


In [15]:
import numpy as np
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

oh_transformer = OneHotEncoder(dtype=np.float64, sparse_output=False, handle_unknown='infrequent_if_exist')
ordinal_transformer = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)



preprocessor=make_column_transformer(
    (Pipeline([
        ('imputer',SimpleImputer(strategy='median'))
    ]),numerical_features
    ),

    (Pipeline([
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('OneHotEncoder',oh_transformer)
    ]), oh_columns),

    (Pipeline([
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('OrdinalEncoder',ordinal_transformer)
    ]),ordinal_columns)
)


In [16]:
X_transformed=preprocessor.fit_transform(X)
X_test_transformed=preprocessor.transform(X_test)

In [27]:

from imblearn.combine import SMOTETomek, SMOTEENN

smt = SMOTEENN(random_state=42,sampling_strategy='minority' )
X_res, y_res = smt.fit_resample(X_transformed, y)

In [142]:
X_res

array([[2.00000000e+01, 6.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [9.00000000e+01, 3.90000000e+01, 4.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.00000000e+00, 3.20000000e+01, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       ...,
       [1.21095262e+02, 3.10476311e+01, 1.97618447e+00, ...,
        9.76184469e-01, 0.00000000e+00, 2.38155315e-02],
       [9.00000000e+01, 1.44715994e+01, 3.82100152e-01, ...,
        3.82100152e-01, 6.17899848e-01, 0.00000000e+00],
       [8.24866377e+00, 1.70000000e+01, 1.00000000e+00, ...,
        3.12165941e-01, 6.87834059e-01, 0.00000000e+00]])

In [144]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score


In [203]:
def evaluate_model(y_true,y_pred):

    f1=f1_score(y_true=y_true,y_pred=y_pred)
    precision=precision_score(y_true=y_true,y_pred=y_pred)
    recall=recall_score(y_true=y_true,y_pred=y_pred)

    return f1,precision,recall

In [204]:
models={

    "RandomForest":RandomForestClassifier(),
    "GradientBoosting":GradientBoostingClassifier()
}


def train_models(X,y,models):


    result={}


    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True)


    for model_name,model in models.items():

        model.fit(X_train,y_train)

        y_train_pred=model.predict(X_train)
        y_test_pred=model.predict(X_test)

        f1_train,precision_train,recall_train=evaluate_model(y_true=y_train,y_pred=y_train_pred)

        f1_test,precision_test,recall_test=evaluate_model(y_true=y_test,y_pred=y_test_pred)

        result[model] = {
            'train': [f1_train, precision_train, recall_train],
            'test': [f1_test, precision_test, recall_test]
        }

    return result


In [205]:
result=train_models(X=X_res,y=y_res,models=models)

In [231]:
for model,_ in result.items():

    y_test_pred=model.predict(X_test_transformed)

    f1,precision,recall=evaluate_model(y_true=y_test,y_pred=y_test_pred)

    print(f"{model}: {f1,precision,recall}")

RandomForestClassifier(): (0.8108996147745298, 0.8126490291813331, 0.8091577162238552)
GradientBoostingClassifier(): (0.7847240665002391, 0.7400320577038669, 0.8351611079706048)


{RandomForestClassifier(): {'train': [1.0, 1.0, 1.0],
  'test': [0.9688295525258811, 0.9739535941765242, 0.9637591446257737]},
 GradientBoostingClassifier(): {'train': [0.9323754358339894,
   0.9239578689255461,
   0.9409477866061294],
  'test': [0.9278212290502793, 0.9211314475873544, 0.9346088913899832]}}

In [19]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
models={

    'RandomForest':{
        'model':RandomForestClassifier(),
        'params':{
            'n_estimators':np.arange(100,100,100),
            'max_depth':np.arange(1,20,1)
        }
    },

    'GradientBoosting':{
        'model':GradientBoostingClassifier(),
        'params':{
            'max_depth':np.arange(1,20,1),
            'n_estimators':np.arange(100,100,100)
        }
    }
}


In [23]:
from hotel_cancellation.utils.main_utils import read_yaml_file

model_configuration=read_yaml_file('../config/model.yaml')
model_configuration

{'models': {'RandomForest': {'model': 'RandomForestClassifier',
   'params': {'n_estimators': {'min': 100, 'max': 1000, 'step': 100},
    'max_depth': {'min': 20, 'max': 100, 'step': 10}}},
  'GradientBoosting': {'model': 'GradientBoostingClassifier',
   'params': {'n_estimators': {'min': 100, 'max': 1000, 'step': 100},
    'max_depth': {'min': 1, 'max': 20, 'step': 1}}}}}

In [24]:
def get_model_instance(model_name):
    if model_name == 'RandomForestClassifier':
        return RandomForestClassifier()
    elif model_name == 'GradientBoostingClassifier':
        return GradientBoostingClassifier()

In [30]:
def param_dict_to_list(param_dict):
    if isinstance(param_dict, dict):
        return list(np.arange(param_dict['min'], param_dict['max'] + param_dict['step'], param_dict['step']))
    return param_dict  

for model_name, model_info in model_configuration['models'].items():
    model = get_model_instance(model_info['model'])
    params = model_info['params']
    
    param_distributions = {key: param_dict_to_list(value) for key, value in params.items()}
    
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        cv=3,
        verbose=1
    )
    
    random_search.fit(X_res, y_res)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


KeyboardInterrupt: 

TypeError: Parameter grid for parameter 'n_estimators' needs to be a list or a numpy array, but got {'min': 100, 'max': 1000, 'step': 100} (of type dict) instead. Single values need to be wrapped in a list with one element.

In [29]:
params

{'n_estimators': {'min': 100, 'max': 1000, 'step': 100},
 'max_depth': {'min': 20, 'max': 100, 'step': 10}}